In [1]:
import os

print(sorted(os.listdir('.')))

['.gitkeeper', '.ipynb_checkpoints', 'os.ipynb']


In [7]:
for dir_name, sub_dirs, files in os.walk('/tmp'):
    print(dir_name)
    print(sub_dirs)
    print(files)

/tmp
['systemd-private-3d1c98679ff2484b8d233f3cf6d15c6b-systemd-resolved.service-y5oldG', '.X11-unix', '.XIM-unix', 'systemd-private-3d1c98679ff2484b8d233f3cf6d15c6b-systemd-timesyncd.service-D3CrRT', 'systemd-private-3d1c98679ff2484b8d233f3cf6d15c6b-rtkit-daemon.service-Hs25bB', '.com.google.Chrome.xp3vve', 'systemd-private-3d1c98679ff2484b8d233f3cf6d15c6b-bolt.service-ZZElYM', 'Temp-f3d620e8-6f36-435d-9639-17b0ba0a25b4', '.Test-unix', '.ICE-unix', 'systemd-private-3d1c98679ff2484b8d233f3cf6d15c6b-fwupd.service-OMsosT', 'systemd-private-3d1c98679ff2484b8d233f3cf6d15c6b-colord.service-T2MWe4', '.font-unix', 'ssh-l04iMwQFuUEB']
['config-err-qqake4']
/tmp/.X11-unix
[]
['X0', 'X1']
/tmp/.XIM-unix
[]
[]
/tmp/.com.google.Chrome.xp3vve
[]
['SingletonCookie', 'SingletonSocket']
/tmp/Temp-f3d620e8-6f36-435d-9639-17b0ba0a25b4
[]
[]
/tmp/.Test-unix
[]
[]
/tmp/.ICE-unix
[]
['1031', '2146']
/tmp/.font-unix
[]
[]
/tmp/ssh-l04iMwQFuUEB
[]
['agent.2146']


In [8]:
for entry in os.scandir('.'):
    if entry.is_dir():
        typ = 'dir'
    elif entry.is_file():
        typ = 'file'
    elif entry.is_symlink():
        typ = 'link'
    else:
        typ = 'unknown'
    print('{name} {typ}'.format(
        name=entry.name,
        typ=typ,
    ))

.gitkeeper file
.ipynb_checkpoints dir
os.ipynb file


In [9]:
stat_info = os.stat('.gitkeeper')

In [10]:
stat_info.st_size

0

In [12]:
oct(stat_info.st_mode)

'0o100644'

In [13]:
stat_info.st_ctime

1538411536.5826325

In [14]:
import stat

filename = 'os_stat_chmod_example.txt'
if os.path.exists(filename):
    os.unlink(filename)
with open(filename, 'wt') as f:
    f.write('contents')

# 使用 stat 函数判断文件当前的权限
existing_permissions = stat.S_IMODE(os.stat(filename).st_mode)

if not os.access(filename, os.X_OK):
    print('Adding execute permission')
    new_permissions = existing_permissions | stat.S_IXUSR
else:
    print('Removing execute permission')
    # 使用 xor 异或清除用户的执行权限
    new_permissions = existing_permissions ^ stat.S_IXUSR

os.chmod(filename, new_permissions)

Adding execute permission


In [15]:
print('Exists:', os.access('.gitkeeper', os.F_OK))
print('Readable:', os.access('.gitkeeper', os.R_OK))
print('Writable:', os.access('.gitkeeper', os.W_OK))
print('Executable:', os.access('.gitkeeper', os.X_OK))

Exists: True
Readable: True
Writable: True
Executable: False


In [17]:
TEST_GID = 502
TEST_UID = 502

def show_user_info():
    print(f"User (actual/effective): {os.getuid()} / {os.geteuid()}")
    print(f"Group (actual/effective): {os.getgid()} / {os.getegid()}")
    print(f"Actual Groups: {os.getgroups()} ")
    
print('Before Change:')
show_user_info()
print()

try:
    os.setegid(TEST_GID) # 使用sudo运行时则可以更改
except OSError:  
    print('Error: could not change effective group')
else:
    print('Change group:')
    show_user_info()
    print()
    
try:
    os.seteuid(TEST_UID)
except OSError:
    print('Error: could not change effective user')
else:
    print('Change user:')
    show_user_info()

Before Change:
User (actual/effective): 1000 / 1000
Group (actual/effective): 1000 / 1000
Actual Groups: [4, 24, 27, 30, 46, 116, 126, 1000] 

Error: could not change effective group
Error: could not change effective user


In [18]:
os.environ['TESTVAR'] = 'this value'
print(os.environ.get('TESTVAR'))

this value


In [23]:
start = os.getcwd()
print('Starting', start)
print('Move to ', os.pardir)
os.chdir(os.pardir)
print('Current', os.getcwd())
os.chdir(start)
print('Move back', os.getcwd())

Starting /home/hejl/practise/standard_library/RuntimeFeature
Move to  ..
Current /home/hejl/practise/standard_library
Move back /home/hejl/practise/standard_library/RuntimeFeature


In [24]:
os.system('pwd')  # run in shell to see correct result

0

In [25]:
os.system('echo $TMPDIR')

0

In [26]:
pid = os.fork()
if pid:
    print('child process in', pid)
else:
    print('parent process')

child process in 9728
parent process
ERROR! Session/line number was not unique in database. History logging moved to new session 16


In [27]:
import signal
import time

def signal_usr1(signum, frame):
    pid = os.getpid()
    print('receive usr1 in process', pid)
    
print('Forking')
child_pid = os.fork()
if child_pid:
    print('Parent: pausing before sending signal')
    time.sleep(1)
    print('Parent signaling ', child_pid)
    os.kill(child_pid, signal.SIGUSR1)
else:
    print('CHILD: Setting up signal handler')
    signal.signal(signal.SIGUSR1, signal_usr1)
    print('CHILD: Pausing to wait for signal')
    time.sleep(5)

CHILD: Setting up signal handler
CHILD: Pausing to wait for signal
Forking
Parent: pausing before sending signal
Parent signaling  9778
receive usr1 in process 9778
ERROR! Session/line number was not unique in database. History logging moved to new session 15


许多计算密集型程序利用多进程来解决Python和全局解释器锁的线程限制。 当启动多个进程来运行单独的任务时，主进程需要等待其中一个或多个进程完成才能启动新的任务，避免服务器过载。 使用 wait() 及其相关函数，有几种不同的方法。

如果不关心哪个子进程会先退出，使用 wait() 。只要有子进程退出，它就会立即返回。

In [28]:
import sys

for i in range(2):
    print(f"Parent {os.getpid()} : Forking {i}")
    worker_pid = os.fork()
    if not worker_pid:
        print(f"Worker {i} starting")
        time.sleep(2+i)
        print(f"Worker {i} finish")
        sys.exit(i)

for i in range(2):
    print('Parent waiting for ', i)
    done = os.wait()
    print('Parent child done ', done)
        

Worker 0 starting
Parent 7334 : Forking 0
Parent 7334 : Forking 1
Worker 1 starting
Parent 7334 : Forking 0
Parent 7334 : Forking 1
Parent waiting for  0
Worker 0 finish


SystemExit: 0

/home/hejl/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Worker 1 finish


SystemExit: 1

/home/hejl/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


ERROR! Session/line number was not unique in database. History logging moved to new session 14


KeyboardInterrupt: 

In [29]:
workers = []
for i in range(2):
    print(f"Parent {os.getpid()} : Forking {i}")
    worker_pid = os.fork()
    if not worker_pid:
        print(f"Worker {i} starting")
        time.sleep(2+i)
        print(f"Worker {i} finish")
        sys.exit(i)
    workers.append(worker_pid)

for pid in workers:
    print('Parent waiting for pid', pid)
    done = os.waitpid(pid,0)
    print('Parent child done ', done)

Worker 1 starting
Worker 0 starting
Parent 7334 : Forking 0
Parent 7334 : Forking 1
Parent waiting for pid 10418
Worker 0 finish


SystemExit: 0

/home/hejl/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Worker 1 finish


SystemExit: 1

/home/hejl/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


KeyboardInterrupt: 

In [32]:
import errno
for num in [errno.ENOENT, errno.EINTR, errno.EBUSY]:
    name = errno.errorcode[num]
    print(f"{num :>2} {name:<6} : {os.strerror(num)}")


 2 ENOENT : No such file or directory
 4 EINTR  : Interrupted system call
16 EBUSY  : Device or resource busy


操作系统定义和 errno 管理的错误码能被 strerror() 转为消息字符串。